## Plotting Tutorial

The Hail plot module allows for easy plotting of data. This notebook contains examples of how to use the plotting functions in this module, many of which can also be found in the first tutorial.

In [1]:
import hail as hl
hl.init()

from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
SLF4J: Class path contains SLF4J bindings targeting slf4j-api versions 1.7.x or earlier.
SLF4J: Ignoring binding found at [jar:file:/Users/irademac/miniconda3/lib/python3.9/site-packages/pyspark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See https://www.slf4j.org/codes.html#ignoredBindings for an explanation.
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.0
SparkUI available at http://wmf28-2bc.broadinstitute.org:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.113-3d7864e0cff2
LOGGING: writing to /Users/irademac/src/hail/hail/python/hail/docs/tutorials/hail-20230425-0932-0.2.113-3d7864e0cff2.log

Loading BokehJS ...

In [2]:
hl.utils.get_1kg('data/')
mt = hl.read_matrix_table('data/1kg.mt')
table = (hl.import_table('data/1kg_annotations.txt', impute=True)
         .key_by('Sample'))
mt = mt.annotate_cols(**table[mt.s])
mt = hl.sample_qc(mt)

mt.describe()

2023-04-25 09:33:01.291 Hail: INFO: downloading 1KG VCF ...
  Source: https://storage.googleapis.com/hail-tutorial/1kg.vcf.bgz
2023-04-25 09:33:02.328 Hail: INFO: importing VCF and writing to matrix table...
2023-04-25 09:33:04.092 Hail: INFO: scanning VCF for sortedness...
2023-04-25 09:33:13.255 Hail: INFO: Coerced sorted VCF - no additional import work to do
2023-04-25 09:33:20.477 Hail: INFO: wrote matrix table with 10879 rows and 284 columns in 16 partitions to data/1kg.mt
2023-04-25 09:33:20.843 Hail: INFO: downloading 1KG annotations ...
  Source: https://storage.googleapis.com/hail-tutorial/1kg_annotations.txt
2023-04-25 09:33:21.015 Hail: INFO: downloading Ensembl gene annotations ...
  Source: https://storage.googleapis.com/hail-tutorial/ensembl_gene_annotations.txt
2023-04-25 09:33:21.408 Hail: INFO: Done!
2023-04-25 09:33:24.983 Hail: INFO: Reading table to impute column types


----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'Population': str
    'SuperPopulation': str
    'isFemale': bool
    'PurpleHair': bool
    'CaffeineConsumption': int32
    'sample_qc': struct {
        dp_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        gq_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        call_rate: float64, 
        n_called: int64, 
        n_not_called: int64, 
        n_filtered: int64, 
        n_hom_ref: int64, 
        n_het: int64, 
        n_hom_var: int64, 
        n_non_ref: int64, 
        n_singleton: int64, 
        n_snp: int64, 
        n_insertion: int64, 
        n_deletion: int64, 
        n_transition: int64, 
        n_transversion: int64, 
        n_star: int64, 
      

2023-04-25 09:33:27.297 Hail: INFO: Finished type imputation
  Loading field 'Sample' as type str (imputed)
  Loading field 'Population' as type str (imputed)
  Loading field 'SuperPopulation' as type str (imputed)
  Loading field 'isFemale' as type bool (imputed)
  Loading field 'PurpleHair' as type bool (imputed)
  Loading field 'CaffeineConsumption' as type int32 (imputed)


### Histogram

The `histogram()` method takes as an argument an aggregated hist expression, as well as optional arguments for the legend and title of the plot.

In [ ]:
dp_hist = mt.aggregate_entries(hl.expr.aggregators.hist(mt.DP, 0, 30, 30))
p = hl.plot.histogram(dp_hist, legend='DP', title='DP Histogram')
show(p)

This method, like all Hail plotting methods, also allows us to pass in fields of our data set directly. Choosing not to specify the `range` and `bins` arguments would result in a range being computed based on the largest and smallest values in the dataset and a default bins value of 50.

In [ ]:
p = hl.plot.histogram(mt.DP, range=(0, 30), bins=30)
show(p)

### Cumulative Histogram

The `cumulative_histogram()` method works in a similar way to `histogram()`.

In [ ]:
p = hl.plot.cumulative_histogram(mt.DP, range=(0,30), bins=30)
show(p)

### Scatter

The `scatter()` method can also take in either Python types or Hail fields as arguments for x and y.

In [ ]:
p = hl.plot.scatter(mt.sample_qc.dp_stats.mean, mt.sample_qc.call_rate, xlabel='Mean DP', ylabel='Call Rate')
show(p)

We can also pass in a Hail field as a `label` argument, which determines how to color the data points.

In [ ]:
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.97))
ab = mt.AD[1] / hl.sum(mt.AD)
filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                        (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (mt.GT.is_hom_var() & (ab >= 0.9)))
mt = mt.filter_entries(filter_condition_ab)
mt = hl.variant_qc(mt).cache()
common_mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)
gwas = hl.linear_regression_rows(y=common_mt.CaffeineConsumption, x=common_mt.GT.n_alt_alleles(), covariates=[1.0])
pca_eigenvalues, pca_scores, _ = hl.hwe_normalized_pca(common_mt.GT)

In [ ]:
p = hl.plot.scatter(pca_scores.scores[0], pca_scores.scores[1],
                    label=common_mt.cols()[pca_scores.s].SuperPopulation,
                    title='PCA', xlabel='PC1', ylabel='PC2', collect_all=True)
show(p)

Hail's downsample aggregator is incorporated into the `scatter()`, `qq()`, and `manhattan()` functions. The `collect_all` parameter tells the plot function whether to collect all values or downsample. Choosing not to set this parameter results in downsampling.

In [ ]:
p2 = hl.plot.scatter(pca_scores.scores[0], pca_scores.scores[1],
                    label=common_mt.cols()[pca_scores.s].SuperPopulation,
                    title='PCA (downsampled)', xlabel='PC1', ylabel='PC2', collect_all=False, n_divisions=50)

show(gridplot([p, p2], ncols=2, plot_width=400, plot_height=400))

### 2-D histogram

For visualizing relationships between variables in large datasets (where scatter plots may be less informative since they highlight outliers), the `histogram_2d()` function will create a heatmap with the number of observations in each section of a 2-d grid based on two variables.

In [ ]:
p = hl.plot.histogram2d(pca_scores.scores[0], pca_scores.scores[1])
show(p)

### Q-Q (Quantile-Quantile)

The `qq()` function requires either a Python type or a Hail field containing p-values to be plotted. This function also allows for downsampling.

In [ ]:
p = hl.plot.qq(gwas.p_value, collect_all=True)
p2 = hl.plot.qq(gwas.p_value, n_divisions=75)

show(gridplot([p, p2], ncols=2, plot_width=400, plot_height=400))

### Manhattan

The `manhattan()` function requires a Hail field containing p-values.

In [ ]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

We can also pass in a dictionary of fields that we would like to show up as we hover over a data point, and choose not to downsample if the dataset is relatively small.

In [ ]:
hover_fields = dict([('alleles', gwas.alleles)])
p = hl.plot.manhattan(gwas.p_value, hover_fields=hover_fields, collect_all=True)
show(p)

## GGPlot Tutorial


In [ ]:
import hail as hl
from hail.ggplot import *

import plotly
import plotly.io as pio
pio.renderers.default='iframe'

The Hail team has implemented a plotting module for hail based on the very popular `ggplot2` package from R's tidyverse. That library is very fully featured and we will never be quite as flexible as it, but with just a subset of its functionality we can make highly customizable plots.

### The Grammar of Graphics

The key idea here is that there's not one magic function to make the plot you want. Plots are built up from a set of core primitives that allow for extensive customization. Let's start with an example. We are going to plot y = x^2 for x from 0 to 10. First we make a hail table representing that data:

In [ ]:
ht = hl.utils.range_table(10)
ht = ht.annotate(squared = ht.idx**2)

Every plot starts with a call to `ggplot`, and then requires adding a `geom` to specify what kind of plot you'd like to create.

In [ ]:
fig = ggplot(ht, aes(x=ht.idx, y=ht.squared)) + geom_line()
fig.show()

`aes` creates an "aesthetic mapping", which maps hail expressions to aspects of the plot. There is a predefined list of aesthetics supported by every `geom`. Most take an `x` and `y` at least. 

With this interface, it's easy to change out our plotting representation separate from our data. We can plot bars:

In [ ]:
fig = ggplot(ht, aes(x=ht.idx, y=ht.squared)) + geom_col()
fig.show()

Or points:

In [ ]:
fig = ggplot(ht, aes(x=ht.idx, y=ht.squared)) + geom_point()
fig.show()

There are optional aesthetics too. If we want, we could color the points based on whether they're even or odd:

In [ ]:
fig = ggplot(ht, aes(x=ht.idx, y=ht.squared, color=hl.if_else(ht.idx % 2 == 0, "even", "odd"))) + geom_point()
fig.show()

Note that the `color` aesthetic by default just takes in an expression that evaluates to strings, and it assigns a discrete color to each string.

Say we wanted to plot the line with the colored points overlayed on top of it. We could try:

In [ ]:
fig = (ggplot(ht, aes(x=ht.idx, y=ht.squared, color=hl.if_else(ht.idx % 2 == 0, "even", "odd"))) + 
       geom_line() + 
       geom_point() 
      )
fig.show()

But that is coloring the line as well, causing us to end up with interlocking blue and orange lines, which isn't what we want. For that reason, it's possible to define aesthetics that only apply to certain geoms.

In [ ]:
fig = (ggplot(ht, aes(x=ht.idx, y=ht.squared)) + 
       geom_line() + 
       geom_point(aes(color=hl.if_else(ht.idx % 2 == 0, "even", "odd"))) 
      )
fig.show()

All geoms can take in their own aesthetic mapping, which lets them specify aesthetics specific to them. And `geom_point` still inherits the `x` and `y` aesthetics from the mapping defined in `ggplot()`.

### Geoms that group

Some geoms implicitly do an aggregation based on the `x` aesthetic, and so don't take a `y` value. Consider this dataset from gapminder with information about countries around the world, with one datapoint taken per country in the years 1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992, 1997, 2002, and 2007.

In [ ]:
gp = hl.Table.from_pandas(plotly.data.gapminder())
gp.describe()

Let's filter the data to 2007 for our first experiments

In [ ]:
gp_2007 = gp.filter(gp.year == 2007)

If we want to see how many countries from each continent we have, we can use `geom_bar`, which just takes in an x aesthetic and then implicitly counts how many values of each x there are.

In [ ]:
ggplot(gp_2007, aes(x=gp_2007.continent)) + geom_bar()

To make it a little prettier, let's color per continent as well. We use `fill` to specify color of shapes (as opposed to `color` for points and lines. `color` on a bar chart sets the color of the bar outline.)

In [ ]:
ggplot(gp_2007, aes(x=gp_2007.continent)) + geom_bar(aes(fill=gp_2007.continent))

Maybe we instead want to see not the number of countries per continent, but the number of people living on each continent. We can do this with `geom_bar` as well by specifying a weight.

In [ ]:
ggplot(gp_2007, aes(x=gp_2007.continent)) + geom_bar(aes(fill=gp_2007.continent, weight=gp_2007.pop))

Histograms are similar to bar plots, except they break a continuous x axis into bins. Let's import the `iris` dataset for this.


In [ ]:
iris = hl.Table.from_pandas(plotly.data.iris())
iris.describe()

Let's make a histogram:

In [ ]:
ggplot(iris, aes(x=iris.sepal_length, fill=iris.species)) + geom_histogram()

By default histogram plots groups stacked on top of each other, which is not always easy to interpret. We can specify the `position` argument to histogram to get different behavior. `"dodge"` puts the bars next to each other:

In [ ]:
ggplot(iris, aes(x=iris.sepal_length, fill=iris.species)) + geom_histogram(position="dodge")

And `"identity"` plots them over each other. It helps to set an `alpha` value to make them slightly transparent in these cases

In [ ]:
ggplot(iris, aes(x=iris.sepal_length, fill=iris.species)) + geom_histogram(position="identity", alpha=0.8)

### Labels and Axes

It's always a good idea to label your axes. This can be done most easily with `xlab` and `ylab`. We can also use `ggtitle` to add a title. Let's pull in the same plot from above, and add labels.

In [ ]:
(ggplot(iris, aes(x=iris.sepal_length, fill=iris.species)) + 
 geom_histogram(position="identity", alpha=0.8) + 
 xlab("Sepal Length") + ylab("Number of samples") + ggtitle("Sepal length by flower type")
)